In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict
import numpy as np

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 3
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年02月04日"  # include
END_DATE = "2023年03月07日"  # include

TARGET_CATEGORY = "夢幻VMAX"

"""
放逐_騎拉帝納VSTAR

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_白馬VMAX
阿爾_鋁鋼龍VMAX
純阿爾VSTAR

夢幻VMAX

密勒頓ex

洛奇亞VSTAR
一擊_洛奇亞

沙奈朵ex
沙奈朵_蒼響
沙奈朵_超夢V-UNION

起源帕路奇亞VSTAR

連擊武道熊師VMAX

達克萊伊VSTAR

洗翠索羅亞克VSTAR

--

飄香豚ex

雷吉鐸拉戈VSTAR

烈空坐VMAX

火焰雞VMAX



起源帝牙盧卡VSTAR	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1
"""

'\n放逐_騎拉帝納VSTAR\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_白馬VMAX\n阿爾_鋁鋼龍VMAX\n純阿爾VSTAR\n\n夢幻VMAX\n\n密勒頓ex\n\n洛奇亞VSTAR\n一擊_洛奇亞\n\n沙奈朵ex\n沙奈朵_蒼響\n沙奈朵_超夢V-UNION\n\n起源帕路奇亞VSTAR\n\n連擊武道熊師VMAX\n\n達克萊伊VSTAR\n\n洗翠索羅亞克VSTAR\n\n--\n\n飄香豚ex\n\n雷吉鐸拉戈VSTAR\n\n烈空坐VMAX\n\n火焰雞VMAX\n\n\n\n起源帝牙盧卡VSTAR\t1\n\n索羅風速狗\t1\n\n洛托姆VSTAR\t1\n\n月石\t1\n\n化石翼龍VSTAR\t1\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['夢幻VMAX', '交錯切換機', '化石翼龍VSTAR', 'LTB', 'LTB_勾魂眼_噴火龍', '雷吉艾勒奇VMAX', '密勒頓ex', 'Other_Lost', 'LTB_洗翠黏美龍VSTAR', '達克萊伊VSTAR', '起源帕路奇亞VSTAR', '白馬蕾冠王VMAX', '放逐_騎拉帝納VSTAR', '超夢 V-UNION', '阿爾宙斯VSTAR', '光輝無極汰那', '飛翔皮卡丘VMAX', '酋雷姆VMAX', '起源帝牙盧卡VSTAR', 'LTB_空之封印石_水超鋼雷', 'LTB_空之封印石_水超闘雷', 'LTB_蓋歐卡', 'LTB_空之封印石_other', 'LTB_空之封印石_超鋼雷', '阿羅拉六尾VSTAR', '阿爾_白馬VMAX', 'LTB_空之封印石_水超', '阿爾_飛天皮VMAX', '連擊武道熊師VMAX', 'LTB_空之封印石_超闘', 'LTB_空之封印石_水超鋼闘', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超雷', 'LTB_快龍V', 'LTB_空之封印石_悪水超鋼', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_草超鋼雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_超闘雷', 'LTB_空之封印石_草超鋼闘雷', 'LTB_空之封印石_超鋼闘雷', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超鋼', 'LTB_空之封印石_草超闘雷', 'LTB_空之封印石_水草超鋼雷', 'LTB_空之封印石_水草超鋼闘雷', 'LTB_空之封印石_悪水超闘', 'LTB_空之封印石_悪水超', '沙奈朵ex', 'LTB_空之封印石_超', '沙奈朵_蒼響', 'LTB_空之封印石_悪水超鋼闘', '洛托姆VSTAR', 'カラマネロVMAX', 'LTB_古月鳥', 'LTB_噴火龍', '阿爾_騎拉帝納VSTAR', '雷吉鐸拉戈VSTAR', '洛奇亞VSTAR', '沙奈朵_超夢V-UNION', '黑馬蕾冠王VMAX', 'Other_騎拉帝納VSTAR', '純阿爾VSTAR', '阿爾_鋁鋼龍VMAX', '烈空坐VMAX', '

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.510185241699219e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_target_deck = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_target_deck = deck_cnt

In [11]:
deck_cnt_list = sorted(deck_cnt_list, reverse=True)
for num_deck, deck_name in deck_cnt_list:
    if num_deck > 0:
        print(f"{deck_name}\t{num_deck}\n")

放逐_騎拉帝納VSTAR	478

LTB	399

密勒頓ex	316

阿爾宙斯VSTAR	312

雷吉艾勒奇VMAX	299

交錯切換機	292

洛奇亞VSTAR	272

夢幻VMAX	258

一擊_洛奇亞	254

飛翔皮卡丘VMAX	156

LTB_勾魂眼_噴火龍	146

Other_騎拉帝納VSTAR	144

沙奈朵ex	120

阿爾_騎拉帝納VSTAR	92

連擊武道熊師VMAX	82

沙奈朵_蒼響	79

LTB_蓋歐卡	68

阿爾_飛天皮VMAX	64

白馬蕾冠王VMAX	57

起源帕路奇亞VSTAR	54

超夢 V-UNION	53

沙奈朵_超夢V-UNION	49

阿爾_鋁鋼龍VMAX	43

洗翠索羅亞克VSTAR	35

阿爾_白馬VMAX	25

黑馬蕾冠王VMAX	19

達克萊伊VSTAR	19

LTB_洗翠黏美龍VSTAR	19

others	18

LTB_空之封印石_水超闘雷	17

LTB_快龍V	17

純阿爾VSTAR	15

Other_Lost	15

LTB_空之封印石_水超鋼闘	13

LTB_空之封印石_other	12

光輝無極汰那	11

起源帝牙盧卡VSTAR	10

火焰雞VMAX	10

LTB_空之封印石_水超鋼雷	9

酋雷姆VMAX	8

洛托姆VSTAR	7

LTB_空之封印石_水超闘	7

飄香豚ex	6

雷吉鐸拉戈VSTAR	5

烈空坐VMAX	5

阿羅拉六尾VSTAR	4

藏飽栗鼠VMAX	4

LTB_空之封印石_水超雷	4

幻影索羅亞克	3

化石翼龍VSTAR	3

LTB_空之封印石_水超	3

LTB_空之封印石_悪水超鋼闘	3

索羅風速狗	2

柱神	2

月石	2

ヒスイ ダイケンキVSTAR	2

LTB_空之封印石_水超鋼闘雷	2

LTB_空之封印石_悪水超闘	2

LTB_空之封印石_悪水超鋼	2

ハピナスV	1

LTB_空之封印石_超闘	1

LTB_空之封印石_超	1

LTB_空之封印石_水超鋼	1

LTB_空之封印石_悪水超雷	1

LTB_空之封印石_悪水超	1

LTB_噴火龍	1



In [12]:
# number of decks for the target category
print(number_target_deck)
if number_target_deck <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

258


In [13]:
def fill_na(col):
    if col.name not in ["date", "prefecture"]:
        return col.fillna(0)
    else:
        return col

In [14]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]
weeks = [
    ("2023年02月04日", "2023年03月10日"),  # total
    ("2023年02月04日", "2023年02月10日"),  # week1 ...
    ("2023年02月11日", "2023年02月17日"),
    ("2023年02月18日", "2023年02月24日"),
    ("2023年02月25日", "2023年03月03日"),
    ("2023年03月04日", "2023年03月10日"),
]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str < real_start_date or date_exclude_jp_str > real_end_date:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.apply(fill_na)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # Calculate for each week
    for week_id, week in enumerate(weeks):
        week_name = f"week_{week_id}" if week_id > 0 else "all"
        week_start, week_end = week
        df_week = df.loc[(df['date'] >= week_start) & (df['date'] <= week_end)]

        # calculate
        num_decks = df_week.shape[0]
        num_used = df_week[col_list].sum(axis="rows", numeric_only=True)
        num_picked = df_week[col_list].astype(bool).sum(axis="rows")
        avg_num_used = num_used / num_picked
        pick_rate = num_picked / num_decks

        # insert rows in df
        avg_idx = f"avg_num_used_{week_name}"
        pick_idx = f"pick_rate_{week_name}"
        df.loc[avg_idx] = {}
        df.loc[pick_idx] = {}
        for col in col_list:
            df.loc[avg_idx, col] = avg_num_used[col] if not np.isnan(avg_num_used[col]) else 0
            df.loc[pick_idx, col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    num_weeks = len(weeks)
    # target_rows = [num_rows - 1, num_rows - 2]
    target_rows = [num_rows - i for i in range(num_weeks*2-1, 0, -2)]
    target_rows += [num_rows - i for i in range(num_weeks*2, 1, -2)]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate_all", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [15]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{real_start_date}-{real_end_date}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.3f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if "avg_num_used" in deck_id or "pick_rate" in deck_id:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

/Users/blake.pan/anaconda3/envs/sandbox_crawler/lib/python3.7/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [16]:
df_list[0]

,date,prefecture,num_players,rank,夢幻V\nS8 039/100,夢幻VMAX\nS8 040/100,蓋諾賽克特V\nS8 069/100,美洛耶塔\nS8 048/100,代歐奇希斯\nS8 045/100,花舞鳥\nS8 017/100,龍王蠍V\nS12a 085/172,光輝甜冷美后\nS11a 009/068,冰砌鵝\nS8b 041/184,古月鳥\nS11 033/100,鑰圈兒\nSV1V 045/078,貪心栗鼠\nSV1V 061/078,光輝基拉祈\nS11a 045/068,大比鳥V\nS11 082/100,多龍梅西亞\nS8 049/100
pick_rate_all,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.422481,0.282946,0.228682,0.027132,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876
pick_rate_week_1,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.625000,0.312500,0.333333,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_2,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.480000,0.300000,0.280000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_3,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.303371,0.202247,0.191011,0.022472,0.000000,0.000000,0.011236,0.011236,0.011236,0.000000,0.000000,0.000000
pick_rate_week_4,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.500000,0.500000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_5,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.0,0.380000,0.300000,0.180000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.020000
avg_num_used_all,NaN,NaN,<NA>,<NA>,3.848837,2.968992,4.0,1.449541,1.452055,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000
avg_num_used_week_1,NaN,NaN,<NA>,<NA>,3.875000,2.958333,4.0,1.600000,1.200000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_2,NaN,NaN,<NA>,<NA>,3.880000,2.960000,4.0,1.583333,1.400000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_3,NaN,NaN,<NA>,<NA>,3.820225,2.977528,4.0,1.370370,1.666667,1.000000,1.000000,0.000000,0.000000,2.000000,3.000000,1.000000,0.000000,0.000000,0.000000


In [17]:
df_list[1]

,date,prefecture,num_players,rank,高級球,強力糖錠,森林封印石,講究腰帶,放逐吸塵器,對戰VIP參加證,機器鶘,巢穴球,離洞繩,朋友手冊,飛羽球,山谷回音喇叭,交替推車,交錯切換機,洗翠的沉重球,寶可夢交替,消災手套,霧之水晶,健行鞋,寶可夢補捉器,潮漩之扇,捕獲香氛,緊急果凍,超級球,消除香水,活力頭帶,野餐籃,全罩防守,寶可裝置3.0,鍋型頭盔,不服輸頭帶,能量籤
pick_rate_all,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,0.992248,0.984496,0.984496,0.961240,0.930233,0.891473,0.786822,0.755814,0.639535,0.558140,0.453488,0.236434,0.228682,0.131783,0.085271,0.081395,0.050388,0.038760,0.019380,0.019380,0.019380,0.011628,0.007752,0.007752,0.007752,0.007752,0.003876,0.003876,0.003876
pick_rate_week_1,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,0.979167,0.958333,1.000000,1.000000,0.937500,0.916667,0.770833,0.729167,0.604167,0.625000,0.270833,0.083333,0.229167,0.083333,0.145833,0.041667,0.020833,0.000000,0.020833,0.041667,0.083333,0.020833,0.020833,0.020833,0.020833,0.000000,0.020833,0.000000,0.000000
pick_rate_week_2,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,0.980000,0.980000,1.000000,0.980000,0.940000,0.860000,0.820000,0.780000,0.560000,0.560000,0.460000,0.220000,0.260000,0.180000,0.120000,0.100000,0.020000,0.000000,0.040000,0.040000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_3,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,1.000000,0.988764,0.977528,0.932584,0.943820,0.876404,0.786517,0.808989,0.629213,0.460674,0.550562,0.224719,0.247191,0.191011,0.056180,0.089888,0.067416,0.033708,0.022472,0.000000,0.000000,0.011236,0.000000,0.000000,0.011236,0.022472,0.000000,0.011236,0.000000
pick_rate_week_4,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.750000,0.750000,0.750000,0.875000,0.750000,0.875000,0.375000,0.375000,0.125000,0.000000,0.000000,0.250000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_5,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,1.000000,1.000000,0.980000,0.940000,0.940000,0.960000,0.780000,0.680000,0.760000,0.580000,0.480000,0.320000,0.200000,0.060000,0.060000,0.060000,0.100000,0.100000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000
avg_num_used_all,NaN,NaN,<NA>,<NA>,3.988372,4.0,2.170543,1.968750,2.988189,3.889764,3.625000,2.325000,1.634783,1.004926,1.600000,1.006061,1.708333,3.982906,1.000000,1.491525,1.000000,2.090909,3.047619,2.230769,1.000000,1.800000,1.000000,2.200000,1.000000,1.500000,1.500000,1.500000,2.000000,1.000000,1.000000,1.000000
avg_num_used_week_1,NaN,NaN,<NA>,<NA>,4.000000,4.0,2.125000,1.957447,2.326087,3.895833,3.375000,2.311111,1.659091,1.000000,1.771429,1.034483,1.900000,4.000000,1.000000,1.454545,1.000000,2.142857,3.500000,2.000000,0.000000,2.000000,1.000000,1.750000,1.000000,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000
avg_num_used_week_2,NaN,NaN,<NA>,<NA>,4.000000,4.0,2.060000,1.938776,2.755102,3.820000,3.653061,2.468085,1.534884,1.024390,1.384615,1.000000,1.785714,4.000000,1.000000,1.384615,1.000000,2.000000,3.000000,3.000000,0.000000,1.500000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_3,NaN,NaN,<NA>,<NA>,3.966292,4.0,2.134831,1.977528,3.306818,3.873563,3.710843,2.392857,1.679487,1.000000,1.638889,1.000000,1.707317,4.000000,1.000000,1.454545,1.000000,2.600000,3.250000,2.166667,1.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000


In [18]:
df_list[2]

,date,prefecture,num_players,rank,老大的指令,裁判,杜娟,小菊兒的璀璨,派帕,莎莉娜,星月,賽寶利,牡丹,皮歐尼,阿克羅瑪的實驗,夏科婭,博士的研究,捩木
pick_rate_all,NaN,NaN,<NA>,<NA>,1.000000,0.972868,0.833333,0.434109,0.197674,0.108527,0.081395,0.050388,0.046512,0.007752,0.003876,0.003876,0.003876,0.003876
pick_rate_week_1,NaN,NaN,<NA>,<NA>,1.000000,0.979167,0.875000,0.625000,0.166667,0.125000,0.125000,0.000000,0.062500,0.020833,0.000000,0.000000,0.000000,0.000000
pick_rate_week_2,NaN,NaN,<NA>,<NA>,1.000000,0.960000,0.820000,0.480000,0.140000,0.140000,0.060000,0.040000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_3,NaN,NaN,<NA>,<NA>,1.000000,0.966292,0.831461,0.325843,0.258427,0.089888,0.067416,0.033708,0.033708,0.000000,0.011236,0.000000,0.000000,0.000000
pick_rate_week_4,NaN,NaN,<NA>,<NA>,1.000000,1.000000,0.750000,0.500000,0.250000,0.000000,0.250000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000
pick_rate_week_5,NaN,NaN,<NA>,<NA>,1.000000,0.980000,0.840000,0.380000,0.160000,0.120000,0.040000,0.140000,0.020000,0.020000,0.000000,0.020000,0.020000,0.000000
avg_num_used_all,NaN,NaN,<NA>,<NA>,1.899225,2.302789,1.009302,2.133929,1.352941,1.107143,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
avg_num_used_week_1,NaN,NaN,<NA>,<NA>,2.000000,2.000000,1.023810,2.066667,1.375000,1.166667,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_2,NaN,NaN,<NA>,<NA>,1.860000,2.187500,1.000000,2.250000,1.142857,1.285714,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_3,NaN,NaN,<NA>,<NA>,1.865169,2.453488,1.000000,2.103448,1.478261,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000


In [19]:
df_list[3]

,date,prefecture,num_players,rank,通頂雪道,放逐市,結晶洞窟,崩塌的競技場,神奧神殿,桌台市,遠古墓地,睿智湖,寶可補給站,冠軍慶典,海灘場地
pick_rate_all,NaN,NaN,<NA>,<NA>,0.895349,0.484496,0.108527,0.062016,0.027132,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876
pick_rate_week_1,NaN,NaN,<NA>,<NA>,0.916667,0.666667,0.187500,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_2,NaN,NaN,<NA>,<NA>,0.880000,0.580000,0.180000,0.020000,0.020000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_3,NaN,NaN,<NA>,<NA>,0.910112,0.393258,0.044944,0.044944,0.011236,0.000000,0.000000,0.011236,0.011236,0.011236,0.011236
pick_rate_week_4,NaN,NaN,<NA>,<NA>,1.000000,0.500000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pick_rate_week_5,NaN,NaN,<NA>,<NA>,0.840000,0.380000,0.100000,0.040000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_all,NaN,NaN,<NA>,<NA>,2.406926,1.424000,1.107143,1.000000,1.285714,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000
avg_num_used_week_1,NaN,NaN,<NA>,<NA>,1.954545,1.562500,1.111111,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_2,NaN,NaN,<NA>,<NA>,2.227273,1.379310,1.222222,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
avg_num_used_week_3,NaN,NaN,<NA>,<NA>,2.641975,1.428571,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,2.000000,1.000000,2.000000


In [20]:
df_list[4]

,date,prefecture,num_players,rank,雙重渦輪能量,匯流能量,基本超能量,連擊能量
pick_rate_all,NaN,NaN,<NA>,<NA>,0.996124,0.434109,0.031008,0.003876
pick_rate_week_1,NaN,NaN,<NA>,<NA>,1.000000,0.625000,0.062500,0.000000
pick_rate_week_2,NaN,NaN,<NA>,<NA>,0.980000,0.480000,0.060000,0.000000
pick_rate_week_3,NaN,NaN,<NA>,<NA>,1.000000,0.325843,0.022472,0.000000
pick_rate_week_4,NaN,NaN,<NA>,<NA>,1.000000,0.500000,0.000000,0.125000
pick_rate_week_5,NaN,NaN,<NA>,<NA>,1.000000,0.380000,0.000000,0.000000
avg_num_used_all,NaN,NaN,<NA>,<NA>,3.622568,3.973214,1.250000,1.000000
avg_num_used_week_1,NaN,NaN,<NA>,<NA>,3.375000,4.000000,1.000000,0.000000
avg_num_used_week_2,NaN,NaN,<NA>,<NA>,3.632653,4.000000,1.666667,0.000000
avg_num_used_week_3,NaN,NaN,<NA>,<NA>,3.719101,3.965517,1.000000,0.000000


In [21]:
len(decks["others"])

91